In [29]:
import datetime
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
import pylab as py

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression

import math
import time
import datetime
from dython.nominal import associations
from dython.nominal import identify_nominal_columns

In [30]:
data = pd.read_csv("final_bike_sharing.csv")

In [31]:
cols = ['tripduration','gender','dist','hour','birthyear','usertype']

In [32]:
df = data[cols]

## Before Imputation the distributaion is 

In [33]:
# for col in  ['tripduration','gender','dist','hour','birthyear']:
#     np.random.seed(19680801)
#     plt.figure(figsize=(20,8))
#     mu = df[col].mean()
#     sigma = df[col].std()
#     x = df[col]
#     num_bins = 500
#     fig, ax = plt.subplots()
#     # the histogram of the data
#     n, bins, patches = ax.hist(x, num_bins, density=True)
#     # add a 'best fit' line
#     y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
#          np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
#     ax.plot(bins, y, '--')
#     ax.set_xlabel(col)
#     ax.set_ylabel('Probability density')
#     ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
#     plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
# #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
# #     ax.set_ylim([0,2.5])
# #     ax.set_xlim([0, 6])
#     # Tweak spacing to prevent clipping of ylabel
#     fig.tight_layout()
#     plt.show()

In [34]:
# for col in df.columns:
#     norm_cdf = scipy.stats.norm.cdf(df[col]) # calculate the cdf - also discrete
#     # plot the cdf
#     sns.lineplot(x=df[col], y=norm_cdf)
#     plt.show()
#     sm.qqplot(df[col], line ='45')
#     py.show()


In [35]:
for col in  ['tripduration','dist','hour','birthyear']:
    mu = df[col].mean()
    sigma = df[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype categorical type mode is ", df['usertype'].mode() ) 
print("gender mode is : " , df['gender'].mode() )

The variable tripduration has mean = 525.9314525266259 and standard deviation =731.0970649533257 
The variable dist has mean = 0.7842291293398377 and standard deviation =0.5421989613484405 
The variable hour has mean = 13.633500453206436 and standard deviation =5.156725828688158 
The variable birthyear has mean = 1980.439108882846 and standard deviation =10.094119857977768 
The variable usertype categorical type mode is  0    Subscriber
dtype: object
gender mode is :  0    male
dtype: object


In [36]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

In [37]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.loc[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.loc[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
        y_train.loc[i] = np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [38]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [39]:
def one_hot_encoding(X_train,X_test):
    categorical_features=identify_nominal_columns(X_train)
    print(categorical_features)
    one_hot_encoded_data_train = pd.get_dummies(X_train, columns = categorical_features)
    one_hot_encoded_data_test = pd.get_dummies(X_test, columns = categorical_features)
    return one_hot_encoded_data_train,one_hot_encoded_data_test

In [40]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [41]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=40)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=40)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train = ampute_nan_values(X_train,variable,perc=40)
        print('Train set after Amputation', X_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test = ampute_nan_values(X_test,variable,perc=40)
        print('Test set after Amputation', X_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train.mode().iloc[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mode().iloc[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train.mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)        
            
    return X_train , y_train, X_test , y_test

In [42]:
def visualize_dist(df, col):
    np.random.seed(19680801)
    plt.figure(figsize=(20,8))
    mu = df[col].mean()
    sigma = df[col].std()
    x = df[col]
    num_bins = 500
    fig, ax = plt.subplots()
    # the histogram of the data
    n, bins, patches = ax.hist(x, num_bins, density=True)
    # add a 'best fit' line
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
         np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    ax.plot(bins, y, '--')
    ax.set_xlabel(col)
    ax.set_ylabel('Probability density')
    ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
    plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
    #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
    #     ax.set_ylim([0,2.5])
    #     ax.set_xlim([0, 6])
    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

In [43]:
features_for_amputation =['gender','dist','hour','birthyear','usertype']

## Calculate mean and std before imputation

In [44]:
X_train, X_test, y_train, y_test = split_dataset(df)

Train (282432, 5) (282432,)
Test (70608, 5) (70608,)


In [45]:
for col in  ['dist','hour','birthyear']:
    mu = X_train[col].mean()
    sigma = X_train[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype has mode ",  X_train['usertype'].mode())
print(f"The variable gender categorical type mode is ", X_train['gender'].mode() )

The variable dist has mean = 0.7848814720776504 and standard deviation =0.54278813457282 
The variable hour has mean = 13.633274558123725 and standard deviation =5.155597162884377 
The variable birthyear has mean = 1980.450076478586 and standard deviation =10.101242979840942 
The variable usertype has mode  0    Subscriber
dtype: object
The variable gender categorical type mode is  0    male
dtype: object


In [46]:
for col in ['dist','hour','birthyear']:
    mu = X_test[col].mean()
    sigma = X_test[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype has mode ",  X_test['usertype'].mode())
print(f"The variable gender categorical type mode is ", X_test['gender'].mode() ) 

The variable dist has mean = 0.7816197583886165 and standard deviation =0.539831786399696 
The variable hour has mean = 13.634404033537276 and standard deviation =5.161274457822756 
The variable birthyear has mean = 1980.3952384998868 and standard deviation =10.065528836178935 
The variable usertype has mode  0    Subscriber
dtype: object
The variable gender categorical type mode is  0    male
dtype: object


In [47]:
cat_gender = ['gender_female', 'gender_male','gender_unknown']
cat_usertype = ['usertype_Customer', 'usertype_Subscriber']

## Imputation for NA ( drop NA values)

In [48]:
model_metrics ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                          "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                              "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  112973
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  28243
X_test Amputed value count of varaible  28243
Simulate Imputation for NA
Train after NA value removal Imputation (169459, 5) (169459,)
Test after NA value removal Imputation (42365, 5) (42365,)
['gender', 'usertype']
2023-02-19 13:00:40.645393
2023-02-19 13:00:40.723018
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  112973
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  28243
X_test Amputed value count of varaible  28243
Simulate Imputation for NA
Train after 

In [49]:
model_metrics

{'gender_female': {'MAE': 279.4612404888523,
  'MSE': 438417.80640484265,
  'RMSE': 662.131260706548,
  'R2': 0.1692674595756104,
  'intercept': 4374.104983318164,
  'Mu_train': 0.21284794552074543,
  'sigma_train': 0.4093222279717101,
  'Mu_test': 0.20955977811873008,
  'sigma_test': 0.4069992475948453},
 'gender_male': {'MAE': 279.4612404888523,
  'MSE': 438417.80640484265,
  'RMSE': 662.131260706548,
  'R2': 0.1692674595756104,
  'intercept': 4374.104983318164,
  'Mu_train': 0.7277866622604878,
  'sigma_train': 0.4451004443893195,
  'Mu_test': 0.7311695975451434,
  'sigma_test': 0.4433568054895338},
 'gender_unknown': {'MAE': 279.4612404888523,
  'MSE': 438417.80640484265,
  'RMSE': 662.131260706548,
  'R2': 0.1692674595756104,
  'intercept': 4374.104983318164,
  'Mu_train': 0.05936539221876678,
  'sigma_train': 0.2363080023050419,
  'Mu_test': 0.059270624336126516,
  'sigma_test': 0.23613329621755969},
 'dist': {'MAE': 279.4612404888523,
  'MSE': 438417.80640484265,
  'RMSE': 662.1

### Bootstrap for drop

In [50]:
bootstrap_data_drop = {}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  112973
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  28243
X_test Amputed value count of varaible  28243
Simulate Imputation for NA
Train after NA value removal Imputation (169459, 5) (169459,)
Test after NA value removal Imputation (42365, 5) (42365,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  112973
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  28243
X_test Amputed value count of varaible  28243
Simulate Imputation for NA
Train after NA value removal Imputation (169459, 5) (169459,)
Test

In [51]:
bootstrap_data_drop

{'gender': {'R2': [0.17070005001673177, 0.18883285533829136],
  'RMSE': [637.4296545949003, 678.9399170761751],
  'intercept': [3575.5575296016805, 5048.507650087573]},
 'dist': {'R2': [0.17070005001673177, 0.18883285533829136],
  'RMSE': [637.4296545949003, 678.9399170761751],
  'intercept': [3575.5575296016805, 5048.507650087573]},
 'hour': {'R2': [0.17070005001673177, 0.18883285533829136],
  'RMSE': [637.4296545949003, 678.9399170761751],
  'intercept': [3575.5575296016805, 5048.507650087573]},
 'birthyear': {'R2': [0.17070005001673177, 0.18883285533829136],
  'RMSE': [637.4296545949003, 678.9399170761751],
  'intercept': [3575.5575296016805, 5048.507650087573]},
 'usertype': {'R2': [0.17070005001673177, 0.18883285533829136],
  'RMSE': [637.4296545949003, 678.9399170761751],
  'intercept': [3575.5575296016805, 5048.507650087573]}}

### Mode Replacement

In [52]:
model_metrics_mode ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                                "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 14:25:50.801796
2023-02-19 14:25:50.891006
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 14:32:25.496725
2023-02-19 14:32:25.591428
Now model is running for varaible ==== hour
Train (282432,

In [53]:
model_metrics_mode

{'gender_female': {'MAE': 278.8510974551261,
  'MSE': 429346.1384727202,
  'RMSE': 655.2450980150254,
  'R2': 0.16704120726722949,
  'intercept': 4793.41261882389,
  'Mu_train': 0.12770861658735555,
  'sigma_train': 0.33376566669779967,
  'Mu_test': 0.1257364604577385,
  'sigma_test': 0.3315544598581087},
 'gender_male': {'MAE': 278.8510974551261,
  'MSE': 429346.1384727202,
  'RMSE': 655.2450980150254,
  'R2': 0.16704120726722949,
  'intercept': 4793.41261882389,
  'Mu_train': 0.8366721901200997,
  'sigma_train': 0.36966514610012585,
  'Mu_test': 0.8387009970541581,
  'sigma_test': 0.3678091224738874},
 'gender_unknown': {'MAE': 278.8510974551261,
  'MSE': 429346.1384727202,
  'RMSE': 655.2450980150254,
  'R2': 0.16704120726722949,
  'intercept': 4793.41261882389,
  'Mu_train': 0.03561919329254475,
  'sigma_train': 0.18533911617899435,
  'Mu_test': 0.03556254248810333,
  'sigma_test': 0.18519809344843596},
 'dist': {'MAE': 296.20309512189993,
  'MSE': 434616.4783781631,
  'RMSE': 659.

### Bootstrap for mode

In [54]:
bootstrap_data_mode = {}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputa

In [55]:
bootstrap_data_mode

{'gender': {'R2': [0.1691901707830331, 0.18268786822947397],
  'RMSE': [650.3096123914648, 682.4723436188491],
  'intercept': [4246.649786411869, 5441.475044045594]},
 'dist': {'R2': [0.15971824289895858, 0.17251226806889364],
  'RMSE': [654.6101303227972, 686.4351136596181],
  'intercept': [3140.304414545859, 4263.276954002484]},
 'hour': {'R2': [0.1695786625147525, 0.1829356446988254],
  'RMSE': [650.1245977606015, 682.406456539414],
  'intercept': [3659.7524913858038, 4788.559955700806]},
 'birthyear': {'R2': [0.16995932295772187, 0.18343271663254868],
  'RMSE': [649.8529710066634, 682.2553338591882],
  'intercept': [2569.35466440465, 3694.690059488141]},
 'usertype': {'R2': [0.12627457333828868, 0.13916408555302634],
  'RMSE': [667.6028374404362, 700.1695416177811],
  'intercept': [2692.94202213028, 3856.411125982192]}}

## Mean Imputation

In [56]:
variables =['dist','hour','birthyear']
model_metrics_mean ={}
for var in variables:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mean
Train after Mean Imputation (282432, 5) (282432,)
Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 15:34:14.566606
2023-02-19 15:34:14.662198
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mean
Train after Mean Imputation (282432, 5) (282432,)
Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 15:36:19.812627
2023-02-19 15:36:19.907901
Now model is running for varaible ==== birthyear
Train (2824

In [57]:
model_metrics_mean

{'dist': {'MAE': 295.01714558934503,
  'MSE': 434475.5535714506,
  'RMSE': 659.1475961963683,
  'R2': 0.15708981601153482,
  'intercept': 3689.3024862841976,
  'Mu_train': 0.9628653976886472,
  'sigma_train': 0.1890919094568335,
  'Mu_test': 0.9636443462497167,
  'sigma_test': 0.18717482836279148},
 'hour': {'MAE': 279.04212612166725,
  'MSE': 429308.5614859001,
  'RMSE': 655.2164233945148,
  'R2': 0.16711410900960366,
  'intercept': 4211.633042191547,
  'Mu_train': 0.9628653976886472,
  'sigma_train': 0.1890919094568335,
  'Mu_test': 0.9636443462497167,
  'sigma_test': 0.18717482836279148},
 'birthyear': {'MAE': 278.70703050458525,
  'MSE': 429092.73348184454,
  'RMSE': 655.0517029073694,
  'R2': 0.1675328290529129,
  'intercept': 4091.1097985525034,
  'Mu_train': 0.9628653976886472,
  'sigma_train': 0.1890919094568335,
  'Mu_test': 0.9636443462497167,
  'sigma_test': 0.18717482836279148}}

### Bootstrap for mean

In [58]:
bootstrap_data_mean ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mean
Train after Mean Imputation (282432, 5) (282432,)
Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simulate Imputation for Mean
Train after Mean Imputation (282432, 5) (282432,)
Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputa

In [59]:
bootstrap_data_mean

{'gender': {'R2': [0.16932342642439624, 0.18283653949113005],
  'RMSE': [650.2380467056266, 682.4107080998679],
  'intercept': [4188.770314082245, 5393.805352169436]},
 'dist': {'R2': [0.16014091543405917, 0.17279389876065615],
  'RMSE': [654.4354545796571, 686.2589578756648],
  'intercept': [3154.576991128764, 4274.898557640832]},
 'hour': {'R2': [0.16987076249976513, 0.18323849052122865],
  'RMSE': [649.9560598906785, 682.2892272510344],
  'intercept': [3668.200100106171, 4799.399342079559]},
 'birthyear': {'R2': [0.17005467742579844, 0.18355200763873408],
  'RMSE': [649.8349338797207, 682.2145637160384],
  'intercept': [3233.7256964463904, 4874.683261514452]},
 'usertype': {'R2': [0.12791666008157482, 0.1405500850312903],
  'RMSE': [667.0458486397224, 699.5111528985966],
  'intercept': [2312.378825536682, 3480.560871646423]}}

## Sampling Replacement Imputation

In [60]:
model_metrics_sample ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                               "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 17:12:06.772367
2023-02-19 17:12:06.868873
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 17:13:33.830353
2023-02-19 17:13:33.924965
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value co

In [61]:
model_metrics_sample

{'gender_female': {'MAE': 279.02172481453476,
  'MSE': 429593.4366547203,
  'RMSE': 655.4337774746739,
  'R2': 0.1665614330788403,
  'intercept': 4871.378731928526,
  'Mu_train': 0.21246530138227962,
  'sigma_train': 0.4090530399986609,
  'Mu_test': 0.20905563108996147,
  'sigma_test': 0.40663708150486894},
 'gender_male': {'MAE': 279.02172481453476,
  'MSE': 429593.4366547203,
  'RMSE': 655.4337774746739,
  'R2': 0.1665614330788403,
  'intercept': 4871.378731928526,
  'Mu_train': 0.728122167459778,
  'sigma_train': 0.4449280589364896,
  'Mu_test': 0.731645139360979,
  'sigma_test': 0.44310643209284317},
 'gender_unknown': {'MAE': 279.02172481453476,
  'MSE': 429593.4366547203,
  'RMSE': 655.4337774746739,
  'R2': 0.1665614330788403,
  'intercept': 4871.378731928526,
  'Mu_train': 0.05941253115794244,
  'sigma_train': 0.236395600979547,
  'Mu_test': 0.0592992295490596,
  'sigma_test': 0.2361855646958514},
 'dist': {'MAE': 304.74973278662077,
  'MSE': 438513.3431488057,
  'RMSE': 662.20

### Boostrap for sample

In [62]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  112973
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  28243
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... birthyear
Train (282432, 5) (282432,)
Test

In [63]:
bootstrap_data_sample

{'gender': {'R2': [0.1688924821131353, 0.182357363047148],
  'RMSE': [650.3980369070466, 682.5836784616381],
  'intercept': [4299.783870135632, 5495.734954410102]},
 'dist': {'R2': [0.154345250929054, 0.1667660375123784],
  'RMSE': [656.8689521063238, 688.6997553671138],
  'intercept': [2847.238950887484, 3984.6087903016437]},
 'hour': {'R2': [0.1695262336908932, 0.18293273862221326],
  'RMSE': [650.1262959677279, 682.4106777541976],
  'intercept': [3656.8296513073724, 4789.90178462845]},
 'birthyear': {'R2': [0.1699509771022386, 0.18344233009223648],
  'RMSE': [649.8742326858828, 682.2454080645925],
  'intercept': [2157.1548229558257, 3390.0733604630973]},
 'usertype': {'R2': [0.10791181341097794, 0.11883382971134493],
  'RMSE': [674.9495941204361, 707.7867090214263],
  'intercept': [2074.800842981768, 3286.513438181037]}}